### Imports

In [1]:
import os
import sys
import random

import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf

### Building Model

#### Downsampling Block

In [2]:
def downsample_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    downsample_1 = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    downsample_2 = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(downsample_1)
    pooling = tf.keras.layers.MaxPool2D((2, 2), (2, 2))(downsample_2)
    return downsample_2, pooling

#### Upsampling Block

In [3]:
def upsample_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    upsample = tf.keras.layers.UpSampling2D((2, 2))(x)
    concat = tf.keras.layers.Concatenate()([upsample, skip])
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

#### Bottleneck Layer

In [4]:
def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    downsample_1 = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    downsample_2 = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(downsample_1)
    return downsample_2

#### Final Model

In [5]:
def simple_unet(filters, inputs):
    p0 = inputs
    c1, p1 = downsample_block(p0, filters[0]) # 128 -> 64
    c2, p2 = downsample_block(p1, filters[1]) # 64 -> 32
    c3, p3 = downsample_block(p2, filters[2]) # 32 -> 16
    c4, p4 = downsample_block(p3, filters[3]) # 16 -> 8

    bn = bottleneck(p4, f[4])

    u1 = upsample_block(bn, c4, f[3]) #8 -> 16
    u2 = upsample_block(u1, c3, f[2]) #16 -> 32
    u3 = upsample_block(u2, c2, f[1]) #32 -> 64
    u4 = upsample_block(u3, c1, f[0]) #64 -> 128

    outputs = tf.keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = tf.keras.models.Model(inputs, outputs, name="Simple_U-Net")
    return model

In [6]:
f = [16, 32, 64, 128, 256]
input_data = tf.keras.layers.Input(shape=(128, 128, 3))
model = simple_unet(filters=f, inputs=input_data)

In [7]:
model.summary()

Model: "Simple_U-Net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 16) 2320        conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 64, 64, 16)   0           conv2d_1[0][0]                   
_______________________________________________________________________________________

### Data 

In [9]:
x_train = np.load('dataset/cell/X_Train.npy')
y_train = np.load('dataset/cell/Y_Train.npy')

In [10]:
print(x_train.shape)
print(y_train.shape)

(670, 128, 128, 3)
(670, 128, 128, 1)


In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train, epochs=1,validation_split=0.1, batch_size=8)

76/76 [==============================] - 94s 1s/step - loss: 0.3627 - accuracy: 0.8833 - val_loss: 0.1653 - val_accuracy: 0.9490


## Polycap Image Segmentation